In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  # !pip install tf-nightly
  pass
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

print(tf.__version__)

In [ ]:
# get data files
# !wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
# !wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv(train_file_path, sep="\t", names=['label', 'message'])
df_test = pd.read_csv(test_file_path, sep="\t", names=['label', 'message'])

y_train = df_train['label']
X_train = df_train['message']
y_test = df_test['label']
X_test = df_test['message']

def turn_label_to_int(label):
  return 0 if label =="ham" else 1

y_train = y_train.map(turn_label_to_int)
y_test = y_test.map(turn_label_to_int)

vectorizer = TfidfVectorizer(max_features=1000)
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(shape=(X_train.shape[1],)))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=2, validation_data=(X_test, y_test))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict(vectorizer.transform([pred_text]))[0][0]
  prediction = [prediction, "ham" if prediction < 0.5 else "spam"]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
